In [150]:
import csv
import copy
import re
from datetime import datetime, timezone

#import xml.etree.ElementTree as ET
#from lxml import etree

# https://docs.python.org/3/library/xml.dom.minidom.html
# https://docs.python.org/3/library/xml.dom.html
#import xml.dom.minidom as MD

# https://lxml.de/tutorial.html
try:
    from lxml import etree
    from lxml.builder import E
    print("running with lxml.etree")
except ImportError:
    import xml.etree.ElementTree as etree
    print("running with Python's xml.etree.ElementTree")


running with lxml.etree


In [151]:
bl_csv = "../vrt/bonalingvo.csv"
bl_list = []

with open(bl_csv, mode='r', encoding='utf-8') as file:
    bl_list = list(csv.reader(file))
    for row in bl_list:
        print(row)  # Jede Zeile wird als Liste ausgegeben


['-ab-', '(-ab-)', '', "Ankaŭ eblas diri '-instalaĵ-', '-uj-'."]
['-al', '(-al)', '', "(se temas pri ostoj aŭ per aliaj vortoj laŭ la signifo en la unuopaj okazoj) prefere diru '-osto' ."]
['-atoro', '(-ator·o)', '', "(laŭ la signifo en la unuopa okazo) prefere diru '-anto', '-ilo', 'maŝino'."]
['-cito', '(-cit·o)', '', "(en sciencaj vortoj) prefere diru 'ĉelo'."]
['-esk-', '(-esk-)', '', "Prefere diru '-ec-'."]
['-eza', '(-ez·a)', '', "(por lingvoj, ekzemple: angleza) prefere diru '-a'."]
['-fila', '(-fil·a)', '', "Prefere diru '-ema'; (ekzemplo: ruso·fil·a -> rus·em·a/rus·am·a) prefere diru '-ama' ."]
['-fono', '(-fon·o)', '', "(ekzemplo: esperanto-fon-o esperanto-parol-ant-o) prefere diru '-parolanto'."]
['-grafo', '(-graf·o)', '', "(en la senco de registra mezurilo) prefere diru 'registrilo'."]
['-i-', '(-i-)', '', "Prefere diru '-uj-'; (por land-nomoj) prefere diru '-land-'; (en scienco kaj teĥniko) prefere diru '-scienc-'."]
['-icido', '(-icid·o)', '', "Prefere diru '-mortigaĵo',

In [152]:
# ŝargu XML-artikolon
art_xml = "../xml/zoolog.xml"

parser = etree.XMLParser(resolve_entities=False)

# Parse the XML file
tree = etree.parse(art_xml,parser)
root = tree.getroot()

# Print the root element
print(f"Root element: {root.tag}")

# Iterate through child elements
for child in root:
    print(f"Tag: {child.tag}, Attributes: {child.attrib}, Text: {child.text}")

Root element: vortaro
Tag: art, Attributes: {'mrk': '$Id: zoolog.xml,v 1.28 2024/07/28 01:49:13 revo Exp $'}, Text: 

Tag: <cyfunction Comment at 0x7fabdd7e7700>, Attributes: <lxml.etree._ImmutableMapping object at 0x7fabddff3af0>, Text: 
$Log$
versio 1.28 2024/07/28 01:49:13
Vito Redsky: + trd pl

versio 1.27 2023/10/21 20:03:08
revo: transskribo per <pr>

Revision 1.23  2018/08/15 22:10:14  revo
Wolfram Diestel: +PV

Revision 1.22  2018/08/13 20:10:16  revo
Wolfram Diestel: +ekz-oj

Revision 1.21  2018/08/10 15:10:15  revo
Michel Cantigneaux: aperas unue en PV (1934), oficiala en 7OA (1958)

Revision 1.20  2015/07/04 08:37:46  revo
aldono de trd cs per skripto de M. von Laer

Revision 1.19  2015/06/28 10:33:41  revo
aldono de trd sk per skripto de M. von Laer

Revision 1.18  2012/08/11 17:10:12  revo



In [153]:
# ŝargu XML-artikolon
## art_xml = "../xml/abel.xml"
## 
## dom = MD.parse(art_xml)  
## 
## root = dom.documentElement
## 
## print(f'radikelemento: {root}')
## 
## # montru ĉiujn id-elmentojn
## for child in root.childNodes:
##     print(f'Child: {child} Tag: {child.localName}')

In [154]:
# ekstraktu radikon kaj kapvortojn de la derivaĵoj
rad = tree.xpath('//rad')
print(f"# radikoj: {len(rad)}")
radiko = rad[0].text
print(radiko)


drvj = tree.xpath('//drv')
print(drvj)


# radikoj: 1
zoolog
[<Element drv at 0x7fabdc4666c0>]


In [155]:
# elprenas kapvorton el drv, anstataŭigante tld per rad
def drv_kap(drv,rad):
    kap = copy.deepcopy(drv.find('kap'))
    tld = kap.find('tld')
    tld.tail = rad + tld.tail if tld.tail else rad
    etree.strip_elements(kap,'tld',with_tail=False)
    print(kap.text)
    return kap.text

# trovas kapvorton en bl_list
def trovu(drvj):
    trovoj = []
    for d in drvj:
        print(d)
        print(etree.tostring(d).decode()[0:100]+"...")
        kap = drv_kap(d,radiko)
        for bl in bl_list:
            if bl[0] == kap:
                trovoj.append(bl)
    return [d,trovoj]

#kap = drv_kap(drvj[0],radiko)

trv = trovu(drvj)
print(trv)

<Element drv at 0x7fabdc4666c0>
<drv mrk="zoolog.0o">
  <kap><tld/>o</kap>
  <snc>
    <dif>
      <ref tip="dif" cel="zoolog1.0isto...
zoologo
[<Element drv at 0x7fabdc4666c0>, [['zoologo', '(zoolog·o)', '7oa', "Ankaŭ eblas diri 'zoologiisto', 'bestosciencisto'. "]]]


In [156]:
# se ni trovis la vorton en la Bonalingvo-listo
# ni aldonos rimarkon en la fino de la koncerna derivaĵo

#def faru_el(nomo,atributoj={},enhavo=""):
#    el = etree.Element(nomo)
#    for atr, val in atributoj.items():
#        el.set(atr,val)
#    if enhavo:
#        el.text = enhavo
#    return el

def last_snc(drv):
    return drv.findall('snc')[:-1]

# kreu unue la rimarkon
drv = trv[0]
rim_txt = "".join(trv[1][0][3])
rim = E('rim',"\n    ",rim_txt,"\n    ",E('fnt',E('bib',"BL")),"\n  ")
rim.tail="\n"
#fnt = faru_el("fnt")
#bib = faru_el("bib",{},"BL")
#fnt.append(bib)
#rim.append(fnt)
last_snc = drv.findall('snc')[-1]
if last_snc:
    last_snc.addnext(rim)
print(etree.tostring(drv).decode())


<drv mrk="zoolog.0o">
  <kap><tld/>o</kap>
  <snc>
    <dif>
      <ref tip="dif" cel="zoolog1.0isto">Zoologiisto</ref>:
      <ekz>
        anka&ubreve; &scirc;arko en akvario povas esti dan&gcirc;era, tion konstatis la
        <tld/>o kaj
        subakvisto <nom>Tadas Laurinavi&ccaron;ius</nom>, kiu zorgas pri
        akvariaj vivuloj
        <fnt>
          <bib>Monato</bib>
          <aut>last</aut>
          <vrk>&Monato;<url ref="/2007/009350.php">&Scirc;arko-&scirc;oko</url></vrk>
        </fnt>;
      </ekz>
      <ekz>
        forpasis la 10an de marto 2012 doktoro <nom>Wim De Smet</nom>,
        flandra <tld/>o
        <fnt>
          <bib>Monato</bib>
          <aut>Cyreen Knockaert</aut>
          <vrk>&Monato;<url ref="/2013/010992.php">Nomenklaturo: &ccirc;u
          kroma esperanta finvenko?</url></vrk>
        </fnt>.
      </ekz>
    </dif>
  </snc><rim>
    Anka&#365; eblas diri 'zoologiisto', 'bestosciencisto'. 
    <fnt><bib>BL</bib></fnt>
  </rim>

  <trd lng="be"

/tmp/ipykernel_40983/620736184.py:25: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if last_snc:


In [162]:
# altigu la eldonnumneron de la artikolo
re_mrk = r'\$Id:\s+([^\.]+)\.xml,v\s+(\d)\.(\d+)\s+(?:\d\d\d\d\/\d\d\/\d\d\s+\d\d:\d\d:\d\d)(.*?)\$'
art = tree.xpath("//art")[0]
art_mrk = art.get("mrk")
print(art_mrk)
match = re.match(re_mrk, art_mrk)
if match:
	filename = match.group(1)
	major = match.group(2)
	minor = int(match.group(3))+1
	rest =  match.group(4)
	now = datetime.now(timezone.utc)
	tstamp = now.strftime("%Y/%m/%d %H:%M:%S")
	version = f"$Id: {filename}.xml,v {major}.{minor} {tstamp}{rest}$"
	print(version)
	art.set("mrk",version)


$Id: zoolog.xml,v 1.28 2024/07/28 01:49:13 revo Exp $
$Id: zoolog.xml,v 1.29 2025/03/23 14:39:16 revo Exp $
